In [7]:
import sys
sys.path.append('../')
import numpy as np

# **SODA Implementation**

The main idea of SODA is to define specific **mechanism**, which can be modeled as an incomplete information game with continuous type (valuation/observation) and action spaces. By discretizing the type and action space, we construct a approximation **game**.
Then we ca apply different learning algorithms **learner** to compute Nash equilibria in the approximation game, which (hopefully) approximate the Bayes-Nash-Equilibrium of the mechanism

## **Mechanism**

Let's us start with a simple mechanism, the *First-Price Sealed Bid (FPSB)*. The FPSB is a single item auction, which is a mechanism we implemented.

In [2]:
from src.mechanism.single_item import SingleItemAuction 

The Single Item Auction is modelled as a Bayesian (Incomplete Information) Game which is defined by
\begin{equation}
    \Gamma = (N, V, A, F, u)
\end{equation}

- $N$ set of bidders (*bidder*)
- $V$ valuation (or observation) space for each bidder *o_space*
- $A$ action space for each bidder *a_space*
- *F* Prior distribution of valuation (types)
- $u_i:V_i \times A \rightarrow \mathbb{R}$ utility function (induced by auction format)

In [32]:
# bidder (if bidders have the same name, e.g. ['1', '1'], we assume that they're symmetric and we learn one strategy for both.
# We use the symmetric version whenever possible,
bidder = ['1', '2']

# the type and action space (interval [0,1]) for each bidder
o_space = {'1': [0, 1], '2': [0,1]}
a_space = {'1': [0, 1], '2': [0,1]}

# prior distribution (i.i.d. uniform)
param_prior = {'distribution': 'uniform'}

# utility function (first price sealed bid with a tie-breaking rule where no one wins at ties)
param_util = {'payment_rule':'first_price', 'tie_breaking': 'lose'}

All this configurations can be found in the *config* directory.

In [33]:
mechanism = SingleItemAuction(bidder, o_space, a_space, param_prior, param_util)

The central aspect of the mechanism is the utility function. If for the first bidder (idx=0) has a valuation of 1.0 and bids 0.6, while her opponent bids 0.3, then she wins with a payoff of 1.0-0.6:

In [34]:
value = np.array([1.0])
bids = np.array([0.6, 0.3])
mechanism.utility(value, bids, idx=0)

array([[0.4]])

### **Game**

Now given the mechanism, we can define a discretized approximation game.
We only have to specify the number of discretization points for valuation (*n*) and action space (*m*) and pass the respective mechanism to the constructor.

In [35]:
from src.game import Game

In [36]:
n = 3
m = 5
game = Game(mechanism, n, m)

The valuation and action space for each bidder is now discretized

In [40]:
game.o_discr['1'], game.a_discr['1']

(array([0. , 0.5, 1. ]), array([0.  , 0.25, 0.5 , 0.75, 1.  ]))

To compute the gradient etc. for the learning algorithm, we compute the utility for all possible bids and valuations beforehand for each bidder

In [42]:
game.get_utility(mechanism)

As we can see here, the computed utility is an array with 5x5x3 entries which corresponds to all possible combinations of bids of both players and observation of player '1'.

In [46]:
game.utility['1'].shape

(5, 5, 3)